In [1]:
import pandas as pd

In [2]:
woreda_mapped_path = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv'
woreda_mapped_df = pd.read_csv(woreda_mapped_path)

In [8]:
woreda_mapped_df = woreda_mapped_df.set_index(['RegionName', 'ZoneName', 'WoredaName'])

In [10]:
woreda_mapped_df.head()

WoredaLat  WoredaLon  \
RegionName  ZoneName    WoredaName                           
Addis Ababa Addis Ababa Addis Ketema   9.034698  38.726558   
                        Akaki Kality   8.897286  38.803770   
                        Arada          9.035681  38.754684   
                        Bole           8.980035  38.839082   
                        Gulele         9.071479  38.744860   

                                                        match_0  \
RegionName  ZoneName    WoredaName                                
Addis Ababa Addis Ababa Addis Ketema  addis ketema kifle ketema   
                        Akaki Kality                      akaki   
                        Arada                arada kifle ketema   
                        Bole                  bole kifle ketema   
                        Gulele              gulele kifle ketema   

                                                                            match_1  \
RegionName  ZoneName    WoredaName                                                    
Addis Ababa Addis Ababa Addis Ketema  addis ketema operational woreda health office   
                        Akaki Kality                                  akaki - kalit   
                        Arada                                          arada woreda   
                        Bole                                                  bolle   
                        Gulele                                              gulelie   

                                                    match_2  \
RegionName  ZoneName    WoredaName                            
Addis Ababa Addis Ababa Addis Ketema  Addis Ketema Sub City   
                        Akaki Kality           akaki kaliti   
                        Arada                Arada Sub City   
                        Bole                  Bole Sub City   
                        Gulele                      gulelle   

                                                        match_3  \
RegionName  ZoneName    WoredaName                                
Addis Ababa Addis Ababa Addis Ketema                        NaN   
                        Akaki Kality  akaki kality kifle ketema   
                        Arada                               NaN   
                        Bole                                NaN   
                        Gulele                          gullele   

                                                    match_4 match_5 match_6  \
RegionName  ZoneName    WoredaName                                            
Addis Ababa Addis Ababa Addis Ketema                    NaN     NaN     NaN   
                        Akaki Kality  Akaki Kaliti Sub City     NaN     NaN   
                        Arada                           NaN     NaN     NaN   
                        Bole                            NaN     NaN     NaN   
                        Gulele              Gulele Sub City     NaN     NaN   

                                     match_7 match_8 match_9 match_10  \
RegionName  ZoneName    WoredaName                                      
Addis Ababa Addis Ababa Addis Ketema     NaN     NaN     NaN      NaN   
                        Akaki Kality     NaN     NaN     NaN      NaN   
                        Arada            NaN     NaN     NaN      NaN   
                        Bole             NaN     NaN     NaN      NaN   
                        Gulele           NaN     NaN     NaN      NaN   

                                     match_11 match_12 match_13 match_14  \
RegionName  ZoneName    WoredaName                                         
Addis Ababa Addis Ababa Addis Ketema      NaN      NaN      NaN      NaN   
                        Akaki Kality      NaN      NaN      NaN      NaN   
                        Arada             NaN      NaN      NaN      NaN   
                        Bole              NaN      NaN      NaN      NaN   
                        Gulele            NaN      NaN      NaN      NaN   

            

In [24]:
lat, lon = woreda_mapped_df.ix['Addis Ababa', 'Addis Ababa', 'Addis Ketema'][['WoredaLat', 'WoredaLon']]

In [26]:
lon

38.726558114195036

<h1> Function to assign Lat Lons at any level in Ethiopia data </h1>

In [110]:
region_mapped_dir = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data'


class latlonHelper(object):
    ''' Class to look up Lat/Lon at every level of the dimension hierachy.
    When a query is returned this information is lost and must be put back before
    ingesting processed_rows.json
    Class properties are files to load = {'woreda_mapped.csv', 'zone_mapped.csv', 'region_mapped.csv'}'''

    def __init__(self, region_mapped_dir):
        self.region_mapped_dir = region_mapped_dir
        self.files_to_load = {'woreda': 'woreda_mapped.csv', 
                              'zone': 'zone_mapped.csv', 
                              'region':'region_mapped.csv'}
        self.load_maps_to_dataframes()

    def load_maps_to_dataframes(self):
        ''' Build a dictionary of dataframes to look up lat lon.
        Keys are the level of the hierarchy.
        Read in the csv files as type string to preserve double precision.'''
        try:
            self.mapped_dataframes = \
                {level: pd.read_csv('%s/%s' % (self.region_mapped_dir, filename), dtype=str)
                 for level, filename in self.files_to_load.iteritems()}
        except IOError:
            raise AssertionError('Lat/Lon lookup dataframes failed to load from \
                                the set of names provided. files_to_load: \
                                %s' % self.files_to_load )
            #print 'The file could not be loaded: %s' % self.files_to_load

    def update_files_to_load(self, files_to_load):
        '''Input a set of files '''
        if isinstance(files_to_load, set):
            self.files_to_load = files_to_load
            self.load_dataframes()
        else:
            print 'The input must be a set of file names.'
        
    def lookup_latlons(granularity_names):
        woreda_lat_lon = self.mapped_dataframes[]



In [109]:
pd.read_csv?

In [107]:
files_to_load = {'woreda': 'woreda_mapped.csv', 
                              'zone': 'zone_mapped.csv', 
                              'region':'region_mapped.csv'}

{level: pd.read_csv('%s/%s' % (region_mapped_dir, filename))
             for level, filename in files_to_load.iteritems()}

{'region':            RegionName  RegionLat  RegionLon        match_0      match_1  \
 0         Addis Ababa   8.980483  38.785538            NaN          NaN   
 1                Afar  12.036445  40.772735          Affar          NaN   
 2              Amhara  11.564952  38.043536  amhara ( 03 )          NaN   
 3   Benishangul-Gumuz  10.502924  35.440307            b/g  beneshangul   
 4           Dire Dawa   9.606269  42.003027      dere dawa     diredewa   
 5            Gambella   7.683850  34.336754          gamb.      gambela   
 6              Harari   9.289660  42.172526          harar       hareri   
 7             Oromiya   7.323567  38.660525         ormoia       oromia   
 8               SNNPR   6.465809  36.806469      s.n.n.p.r         snnp   
 9              Somali   6.932778  43.329883       somaliya          NaN   
 10             Tigray  13.777115  38.438673            NaN          NaN   
 
              match_2            match_3       match_4      match_5  \
 0   

In [111]:
latlons = latlonHelper(region_mapped_dir)

In [151]:
latlon_maps

,GeoKey,RegionLat,RegionLon,RegionName,WoredaLat,WoredaLon,WoredaName,ZoneLat,ZoneLon,ZoneName,...,match_14,match_15,match_2,match_3,match_4,match_5,match_6,match_7,match_8,match_9
0,addis ababa__addis ababa__addis ketema__,NaN,NaN,Addis Ababa,9.034698,38.726558,Addis Ketema,NaN,NaN,Addis Ababa,...,NaN,NaN,Addis Ketema Sub City,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,addis ababa__addis ababa__akaki kality__,NaN,NaN,Addis Ababa,8.897286,38.803770,Akaki Kality,NaN,NaN,Addis Ababa,...,NaN,NaN,akaki kaliti,akaki kality kifle ketema,Akaki Kaliti Sub City,NaN,NaN,NaN,NaN,NaN
2,addis ababa__addis ababa__arada__,NaN,NaN,Addis Ababa,9.035681,38.754684,Arada,NaN,NaN,Addis Ababa,...,NaN,NaN,Arada Sub City,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,addis ababa__addis ababa__bole__,NaN,NaN,Addis Ababa,8.980035,38.839082,Bole,NaN,NaN,Addis Ababa,...,NaN,NaN,Bole Sub City,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,addis ababa__addis ababa__gulele__,NaN,NaN,Addis Ababa,9.071479,38.744860,Gulele,NaN,NaN,Addis Ababa,...,NaN,NaN,gulelle,gullele,Gulele Sub City,NaN,NaN,NaN,NaN,NaN
5,addis ababa__addis ababa__kirkos__,NaN,NaN,Addis Ababa,9.001004,38.758437,Kirkos,NaN,NaN,Addis Ababa,...,NaN,NaN,chirkos,kirkos kifle ketema,kirkose,Kirkos Sub City,NaN,NaN,NaN,NaN
6,addis ababa__addis ababa__kolfe keraniyo__,NaN,NaN,Addis Ababa,9.010225,38.687197,Kolfe Keraniyo,NaN,NaN,Addis Ababa,...,NaN,NaN,kolfe keranio kifle ketema,kolfe keraniyo kefle ketema,kolfe youth centre,kolfie keranio,Kolfe Keraniyo Sub City,NaN,NaN,NaN
7,addis ababa__addis ababa__lideta__,NaN,NaN,Addis Ababa,9.011481,38.729877,Lideta,NaN,NaN,Addis Ababa,...,NaN,NaN,Lideta Sub City,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,addis ababa__addis ababa__nefas silk lafto__,NaN,NaN,Addis Ababa,8.952101,38.728847,Nefas Silk Lafto,NaN,NaN,Addis Ababa,...,NaN,NaN,nefas silk lafto kifle ketema,nefas silk-lafto,nifas silk,nifas silk lafto,nifas silk lafto woreda,nifase silk lafto,Nefas Silk-Lafto Sub City,NaN
9,addis ababa__addis ababa__yeka__,NaN,NaN,Addis Ababa,9.048984,38.827314,Yeka,NaN,NaN,Addis Ababa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
'ZoneName'.split('Name')

['Zone', '']

In [240]:
# Generate the geo_key given the available column names for woreda, zone, region.

mapped_filenames = ['woreda_mapped.csv', 'zone_mapped.csv', 'region_mapped.csv']

COLUMNS = ['RegionName', 'ZoneName', 'WoredaName']
level_map = {1: 'RegionKey', 2: 'ZoneKey', 3: 'WoredaKey'}


def make_geokey(rwz_list):
    ''' Input a list of region, zone, woreda name.'''
    if len(rwz_list) > 0:
        end_dash = '__' * (4 - len(rwz_list))
        return '%s%s' % ('__'.join(rwz_list).lower(), end_dash)
    else:
        return ''

def process_geokeys(df_mapped):
    name_cols = [column for column in df_mapped.columns if 'Name' in column]
    df_mapped['GeoKey'] = \
        df_mapped[name_cols].apply(lambda x: make_geokey(x), axis=1)
        
    print name_cols
    for level_num, column in enumerate(name_cols):
        print level_num, column
        levels = name_cols[:level_num + 1]
        levelkey = '%sKey' % column.split('Name')[0]
        print levelkey
        df_mapped[levelkey] = \
            df_mapped[levels].apply(lambda x: make_geokey(x), axis=1)
    
    return df_mapped

 
def selfjoin_fill_latlons(dataframe):
    POSTFIX_LIST = ['Key', 'Lat', 'Lon']
    for level in ['Region', 'Zone']:
        level_key = '%sKey' % level
        column_list = ['%s%s' % (level, postfix) for postfix in POSTFIX_LIST]
        dataframe = pd.merge(dataframe[column_list].dropna(), dataframe, how='right', on=level_key, suffixes={'', '_y'})
    # Clean up.
    columns_to_drop = [column for column in dataframe.columns if '_y' in column]
    dataframe.drop(columns_to_drop, axis=1, inplace=True)
    return dataframe
    
    
# Initialize empty dataframe. Then loop through all mapped.csv files and append to it.
latlon_maps = pd.DataFrame()


for filename in mapped_filenames:
    df_mapped = pd.read_csv('%s/%s' % (region_mapped_dir, filename), dtype=str)
    match_columns = [column for column in df_mapped.columns if 'match' in column.lower()]
    df_mapped.drop(match_columns, axis=1, inplace=True)
    df_mapped = process_geokeys(df_mapped)
    latlon_maps = latlon_maps.append(df_mapped, ignore_index=True)

latlon_maps = selfjoin_fill_latlons(latlon_maps)
latlon_maps.set_index('GeoKey', inplace=True)

['RegionName', 'ZoneName', 'WoredaName']
0 RegionName
RegionKey
1 ZoneName
ZoneKey
2 WoredaName
WoredaKey
['RegionName', 'ZoneName']
0 RegionName
RegionKey
1 ZoneName
ZoneKey
['RegionName']
0 RegionName
RegionKey


In [242]:
latlon_maps

,ZoneKey,ZoneLat,ZoneLon,RegionKey,RegionLat,RegionLon,RegionName,WoredaKey,WoredaLat,WoredaLon,WoredaName,ZoneName
GeoKey,,,,,,,,,,,,
addis ababa__addis ababa__addis ketema__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__addis ketema__,9.034698212854872,38.726558114195036,Addis Ketema,Addis Ababa
addis ababa__addis ababa__akaki kality__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__akaki kality__,8.89728556566618,38.803769890705,Akaki Kality,Addis Ababa
addis ababa__addis ababa__arada__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__arada__,9.035681311338866,38.7546835191774,Arada,Addis Ababa
addis ababa__addis ababa__bole__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__bole__,8.98003533307717,38.83908245092445,Bole,Addis Ababa
addis ababa__addis ababa__gulele__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__gulele__,9.071479416494498,38.744859937113176,Gulele,Addis Ababa
addis ababa__addis ababa__kirkos__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__kirkos__,9.001003803530361,38.75843721783356,Kirkos,Addis Ababa
addis ababa__addis ababa__kolfe keraniyo__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__kolfe keraniyo__,9.010224654632118,38.68719697668988,Kolfe Keraniyo,Addis Ababa
addis ababa__addis ababa__lideta__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__lideta__,9.011481143879871,38.729876963602074,Lideta,Addis Ababa
addis ababa__addis ababa__nefas silk lafto__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__nefas silk lafto__,8.952101351535061,38.72884748276097,Nefas Silk Lafto,Addis Ababa


<h1> Lookup geokey and return a dictionary with with lat lon info </h1>

In [252]:
# Input a geokey. Output a dictionary wit lat lons will be returned as a string.

latlons = ['RegionLat', 'RegionLon', 'ZoneLat', 'ZoneLon', 'WoredaLat', 'WoredaLon']
latlon_maps.loc['tigray______'][latlons].dropna().to_dict()

{'RegionLat': '13.777114742142253', 'RegionLon': '38.43867332133363'}

In [ ]:
alternative is to search with region, zone, woreda name in each file separately